# Notebook of scripts to import and parse data

['year', 'constituency', 'constituency_type', 'candidates', 'party', 'vote_count', 'vote_percentage']

## Importing data from CSV

In [1]:
import pandas as pd
import os
import json
import csv

if not os.path.exists("output"):
    os.makedirs("output")

data = pd.read_csv("./RAW DATA/parliamentary-general-election-results-by-candidate.csv")

df = data.loc[data["year"]==1955].drop(["year", "constituency_type"], axis=1)
df["vote_percentage"] = df["vote_percentage"].apply(pd.to_numeric, errors='coerce').apply(lambda x: round(x*100, 2))

df.head()

ImportError: No module named pandas

## winners1955.json
```
{
    "Havelock": {
        "candidates": "Lim Yew Hock",
        "party": "SLF",
        "vote_percentage": "0.8648"
    },
    "Tanjong Pagar": {
        "candidates": "Lee Kuan Yew",
        "party": "PAP",
        "vote_percentage": "0.7833"
    },
```

In [2]:
def get_winners(group):
    return group.loc[group['vote_percentage'] == group['vote_percentage'].max()]

winners = df.groupby('constituency').apply(get_winners).reset_index(drop=True).sort_values(by="vote_percentage", ascending=False).set_index(keys="constituency")

winners.to_json(r"output/winners1955.json", orient="index")
winners.head()

NameError: name 'df' is not defined

I tried using pandas because I hoped it would provide an easier way to create subsets of data. It didn't. The way I did this is less efficient, but the data set is small.


## constituencies1955.json

In [3]:
import csv
import json

keys = ['year', 'constituency', 'constituency_type', 'candidates', 'party', 'vote_count', 'vote_percentage'];
dict = {};
sortedDict = {};

with open('./RAW DATA/parliamentary-general-election-results-by-candidate.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if row[0] == "1955": 
            if row[1] in dict:                
                dict[row[1]].insert(1, {
                    "candidates": row[3],
                    "party": row[4],
                    "vote_count": int(row[5]),
                    "vote_percentage": str(round(float(row[6])*100,2)).format('.2f')
                })
                    
            else:
                dict[row[1]] = [{
                    "candidates": row[3],
                    "party": row[4],
                    "vote_count": int(row[5]),
                    "vote_percentage": str(round(float(row[6])*100,2)).format('.2f')
                }]
                
                
    for district in dict:
        sortedDict[district] = sorted( dict[district], key = lambda i: i["vote_count"],reverse=True)

text_file = open(r"output/constituencies1955.json", "w")
text_file.write(json.dumps(sortedDict))
text_file.close()

# electedGovMembers1955.json
# electedOppMembers1955.json

In [37]:
import csv
import json

with open('output/winners1955.json', 'r') as f:
    winners_dict = json.load(f)
    
    s = sorted( winners_dict, key = lambda k: float
               (winners_dict[k]["vote_percentage"]),reverse=True)
    
gov_winners=[]
opp_winners=[]
parties_in_power = ["SLF", "SA"]
    
for i in s:
    if winners_dict[i]["party"] in parties_in_power:
        gov_winners.append([i, winners_dict[i]])
    else:
        opp_winners.append([i, winners_dict[i]])

text_file = open(r"output/electedGovMembers1955.json", "w")
text_file.write(json.dumps(gov_winners))
text_file.close()
    
text_file = open(r"output/electedOppMembers1955.json", "w")
text_file.write(json.dumps(opp_winners))
text_file.close()
